In [3]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)

2.0.0-alpha0


In [45]:
class ConvBNRelu(keras.layers.Layer):
    def __init__(self, channel, kernel_size=1, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__(name='conv_block')

        self.model = keras.models.Sequential([
            keras.layers.Conv1D(channel, kernel_size, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])

    def call(self, x, training=None):
        x = self.model(x, training=training)

        return x


class InceptionBlk(keras.layers.Layer):

    def __init__(self, channel, strides=1):
        super(InceptionBlk, self).__init__(name='inception_block')

        self.channel = channel
        self.strides = strides

        self.conv1_1 = ConvBNRelu(channel, strides=1)

        self.conv2_1 = ConvBNRelu(channel, strides=1)

        self.conv2_2 = ConvBNRelu(channel, kernel_size=3, strides=strides)

        self.conv3_1 = ConvBNRelu(channel, strides=1)
        self.conv3_2 = ConvBNRelu(channel, kernel_size=3, strides=strides)
        self.conv3_3 = ConvBNRelu(channel, kernel_size=3, strides=strides)

        self.pool = keras.layers.MaxPooling1D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(channel, strides=1)

    def call(self, x):
        # branch 1
        x1 = self.conv1_1(x)

        # branch 2
        x2_1 = self.conv2_1(x)
        x2 = self.conv2_2(x2_1)

        # branch 3
        x3_1 = self.conv3_1(x)
        x3_2 = self.conv3_2(x3_1)
        x3 = self.conv3_3(x3_2)

        # branch 4
        x4 = self.pool(x)
        x4 = self.pool_conv(x4)

        # concat along axis=channel
        x = tf.concat([x1, x2, x3, x4], axis=-1)

        return x

class ReductionBlk(keras.layers.Layer):

    def __init__(self, channel, strides=1):
        super(ReductionBlk, self).__init__(name='reduction_block')

        self.channel = channel
        self.strides = strides

        self.conv1_1 = ConvBNRelu(channel, kernel_size=3, strides=strides)

        self.conv2_1 = ConvBNRelu(channel, strides=1)
        self.conv2_2 = ConvBNRelu(channel, kernel_size=3, strides=1)
        self.conv2_3 = ConvBNRelu(channel, kernel_size=3, strides=strides)

        self.pool = keras.layers.MaxPooling1D(3, strides=strides, padding='same')


    def call(self, x):
        # branch 1
        x1 = self.conv1_1(x)

        # branch 2
        x2_1 = self.conv2_1(x)
        x2_2 = self.conv2_2(x2_1)
        x2 = self.conv2_3(x2_2)

        x3 = self.pool(x)


        # concat along axis=channel
        x = tf.concat([x1, x2, x3], axis=-1)

        return x

class TSCNet(keras.Model):
    def __init__(self, input_shape, num_classes, num_layers, init_channel=16, **kwargs):
        super(TSCNet, self).__init__(**kwargs)
        self.in_channel = init_channel
        self.out_channel = init_channel
        self.num_layers = num_layers
        #self.input_shape = input_shape
        self.num_classes = num_classes
        self.init_channel = init_channel

        self.conv1 = ConvBNRelu(init_channel)
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(self.num_layers):

            for layer_id in range(2):
                #print('layer_id: ', layer_id)
                if layer_id == 0:
                    block = InceptionBlk(self.out_channel, strides=1)
                else:
                    block = ReductionBlk(self.out_channel, strides=2)
                self.blocks.add(block)
            # enlarger out_channels per block
            #self.out_channel *= 2
        




        self.avg_pool = keras.layers.GlobalAveragePooling1D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, x):
        out = self.conv1(x)

        out = self.blocks(out)
        out = self.avg_pool(out)
        out = self.fc(out)

        return out

In [48]:
model = TSCNet((176, 1), 137, 12)
# derive input shape for every layers.
model.build(input_shape=(None, 176, 1))
model.summary()

Model: "tsc_net_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block (ConvBNRelu)      multiple                  96        
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  174400    
_________________________________________________________________
global_average_pooling1d_11  multiple                  0         
_________________________________________________________________
dense_28 (Dense)             multiple                  13289     
Total params: 187,785
Trainable params: 183,529
Non-trainable params: 4,256
_________________________________________________________________


In [43]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv1D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x 
    
        
        


# In[5]:


class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides
        
        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling1D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
        
        
    def call(self, x, training=None):
        
        
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)
                
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
                
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=-1)
        
        return x


# In[6]:


class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=32, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            #self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling1D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out    
            

In [44]:
model = Inception(6, 37)
# derive input shape for every layers.
model.build(input_shape=(None, 176, 1))
model.summary()

Model: "inception_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu_1001 (ConvBNRel multiple                  256       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  599424    
_________________________________________________________________
global_average_pooling1d_8 ( multiple                  0         
_________________________________________________________________
dense_25 (Dense)             multiple                  4773      
Total params: 604,453
Trainable params: 600,549
Non-trainable params: 3,904
_________________________________________________________________


In [37]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x 
    
        
        


# In[5]:


class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides
        
        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
        
        
    def call(self, x, training=None):
        
        
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)
                
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
                
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=3)
        
        return x


# In[6]:


class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            #self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out    
            
        

In [38]:
model = Inception(6, 10)
# derive input shape for every layers.
model.build(input_shape=(None, 28, 28, 1))
model.summary()

Model: "inception_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu_757 (ConvBNRelu multiple                  224       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  447168    
_________________________________________________________________
global_average_pooling2d_16  multiple                  0         
_________________________________________________________________
dense_21 (Dense)             multiple                  650       
Total params: 448,042
Trainable params: 446,090
Non-trainable params: 1,952
_________________________________________________________________
